In [ ]:
import os
import csv
import sqlite3
import itertools
import time
import math

import igraph
import scipy.special

In [ ]:
path_csv = os.path.abspath(os.path.expanduser(input('Read to address csv file: ').strip()))

In [ ]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

def create_node_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Node (
                   addr INTEGER PRIMARY KEY);''')

def insert_nodes(nodes):
    cur.executemany('''INSERT OR IGNORE INTO Node (
                       addr) VALUES (
                       ?);''', nodes)
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')

def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()

def synchronous(mode):
    sql = f'PRAGMA synchronous = {mode}'
    cur.execute(sql)
    conn.commit()
    
index_db = 'index.db'
index_conn = sqlite3.connect(f'file:{index_db}?mode=ro')
index_cur = index_conn.cursor()

def get_addrid(addr):
    index_cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (addr,))
    return index_cur.fetchone()[0]

def attach_indexdb():
    cur.execute('''ATTACH DATABASE ? AS other;''', (index_db,))

In [ ]:
create_node_table()
attach_indexdb()
synchronous('NORMAL')
journal_mode('WAL')

In [ ]:
stime = time.time()
nodes = list()
with open(path_csv, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        addrid = get_addrid(row['Address'])
        nodes.append((addrid,))
insert_nodes(nodes)
etime = time.time()
print(f'Nodes: {len(nodes)} during {etime-stime}')

In [ ]:
stime = time.time()
cur.execute('''SELECT TXI.addr, TXO.addr FROM (
                       SELECT other.TxIn.tx AS tx, Node.addr AS addr FROM Node
                       INNER JOIN other.TxIn ON other.TxIn.addr = Node.addr) AS TXI
               INNER JOIN (SELECT other.TxOut.tx AS tx, Node.addr AS addr FROM Node
                       INNER JOIN other.TxOut ON other.TxOut.addr = Node.addr) AS TXO
                   ON TXI.tx = TXO.tx;''')
edges = cur.fetchall()
etime = time.time()
print(f'Edges: {len(edges)} during {etime-stime}')

In [ ]:
g = igraph.Graph()
g.add_vertices([str(x[0]) for x in nodes])
g.add_edges([(str(x[0]), str(x[1])) for x in edges])
g.write_pickle('graph.igraph')

In [ ]:
stime = time.time()
partition = g.community_leiden(objective_function='modularity')
etime = time.time()
print(f'{len(partition)} during {etime-stime}')

In [ ]:
# # It takes a long time!
# stime = time.time()
# layout = g.layout_drl()
# igraph.plot(g, 'graph.svg', layout=layout)
# etime = time.time()
# print(f'Plotting done during {etime-stime}')

In [ ]:
conn.close()
index_conn.close()